In [5]:
import numpy as np
import imutils
import pickle
import cv2
import os
import tkinter as tk
from tkinter import messagebox
import pandas as pd
import datetime

In [6]:
# Load models and utilities
def load_face_detector(proto_path, model_path):
    """Load the face detector model."""
    print("Loading Face Detector...")
    return cv2.dnn.readNetFromCaffe(proto_path, model_path)

def load_face_embedder(model_path):
    """Load the face embedding model."""
    print("Loading Face Recognizer...")
    return cv2.dnn.readNetFromTorch(model_path)

def load_recognizer_and_label_encoder(recognizer_path, le_path):
    """Load the face recognition model and label encoder."""
    with open(recognizer_path, "rb") as f:
        recognizer = pickle.load(f)
    with open(le_path, "rb") as f:
        le = pickle.load(f)
    return recognizer, le

In [8]:
proto_path = "C:\\Users\\tdun\\Documents\\QuanLyNhanSu\\face-recognition-using-opencv\\model\\face_detection_model\\deploy.prototxt"
proto_path

'C:\\Users\\tdun\\Documents\\QuanLyNhanSu\\face-recognition-using-opencv\\model\\face_detection_model\\deploy.prototxt'

In [9]:
model_path = "C:\\Users\\tdun\\Documents\\QuanLyNhanSu\\face-recognition-using-opencv\\model\\face_detection_model\\res10_300x300_ssd_iter_140000.caffemodel"
model_path

'C:\\Users\\tdun\\Documents\\QuanLyNhanSu\\face-recognition-using-opencv\\model\\face_detection_model\\res10_300x300_ssd_iter_140000.caffemodel'

In [10]:
detector = load_face_detector(proto_path, model_path)
detector

Loading Face Detector...


< cv2.dnn.Net 0000013853023030>

In [11]:
embedder = load_face_embedder('C:\\Users\\tdun\\Documents\\QuanLyNhanSu\\face-recognition-using-opencv\\model\\face_detection_model\\openface_nn4.small2.v1.t7')
recognizer, le = load_recognizer_and_label_encoder('C:\\Users\\tdun\\Documents\\QuanLyNhanSu\\face-recognition-using-opencv\\model\\embeddings.pickle', 'C:\\Users\\tdun\\Documents\\QuanLyNhanSu\\face-recognition-using-opencv\\model\\le.pickle')

Loading Face Recognizer...


In [19]:
recognizer['embeddings']

[array([-0.02556043,  0.20934817, -0.00132326,  0.03267487,  0.05335502,
         0.20238957,  0.06707615, -0.10237139,  0.03820135,  0.02232789,
         0.04711089, -0.04501376, -0.01398812, -0.1707689 ,  0.05151427,
         0.00412635, -0.15958422,  0.03199045, -0.05107535, -0.05263042,
         0.11490177, -0.05249917, -0.07000981,  0.00407747,  0.11519284,
        -0.05046431, -0.13607877, -0.12653306,  0.13495678,  0.09358794,
         0.13844897,  0.09134863, -0.07560939,  0.04308975,  0.13982774,
         0.05416331, -0.00646144, -0.0578406 ,  0.06466474, -0.12314184,
         0.11445823, -0.17525227,  0.05884299,  0.0994765 , -0.02428823,
         0.02304189,  0.11202657,  0.06893364, -0.19335605, -0.11114626,
        -0.07314852,  0.05827302,  0.01296653, -0.09946534,  0.04895112,
         0.09583493, -0.14580522,  0.07463784, -0.10085799, -0.09361353,
        -0.11816856, -0.02602588,  0.19725601, -0.11960459,  0.1261273 ,
         0.10496017, -0.00260025, -0.07874604, -0.1

In [4]:
recognizer['names']

NameError: name 'recognizer' is not defined

In [22]:
print(le)

LabelEncoder()


In [ ]:
def recognize_faces(image, faces, embedder, recognizer, le, attendance_list):
    """Recognize faces in the image and draw the results."""
    for (face, (startX, startY, endX, endY)) in faces:
        (fH, fW) = face.shape[:2]

        # Ensure the face width and height are sufficiently large
        if fW < 20 or fH < 20:
            continue

        # Construct a blob for the face ROI
        face_blob = cv2.dnn.blobFromImage(face, 1.0 / 255, (96, 96),
                                          (0, 0, 0), swapRB=True, crop=False)
        embedder.setInput(face_blob)
        vec = embedder.forward()

        # Perform classification to recognize the face
        preds = recognizer.predict_proba(vec)[0]
        j = np.argmax(preds)
        proba = preds[j]
        name = le.classes_[j]

        # Draw the bounding box of the face along with the associated probability
        text = "{}: {:.2f}%".format(name, proba * 100)
        y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.rectangle(image, (startX, startY), (endX, endY), (0, 0, 255), 2)
        cv2.putText(image, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)

    return image

In [ ]:
def detect_faces(image, detector, min_confidence=0.5):
    """Detect faces in an image."""
    (h, w) = image.shape[:2]
    image_blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300),
                                       (104.0, 177.0, 123.0), swapRB=False, crop=False)
    detector.setInput(image_blob)
    detections = detector.forward()

    faces = []
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        # Filter out weak detections
        if confidence > min_confidence:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # Ensure bounding box is within image dimensions
            startX, startY = max(0, startX), max(0, startY)
            endX, endY = min(w, endX), min(h, endY)

            face = image[startY:endY, startX:endX]
            faces.append((face, (startX, startY, endX, endY)))

    return faces

In [ ]:
cap = cv2.VideoCapture(0)
ret, frame = cap.read()

Press Enter to capture
Press Enter to capture
Press Enter to capture
Press Enter to capture
Press Enter to capture
Press Enter to capture
Press Enter to capture
Press Enter to capture
Press Enter to capture
Press Enter to capture


In [ ]:
attendance_list = {}

# Detect faces in the frame
faces = detect_faces(frame, detector)

# Recognize faces in the frame
output_frame = recognize_faces(frame, faces, embedder, recognizer, le, attendance_list)

# Display the resulting frame
cv2.imshow('Real-Time Face Recognition', output_frame)

NameError: name 'imutils' is not defined